In [1]:
%%time
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , precision_recall_curve , f1_score
from sklearn.metrics import classification_report , roc_auc_score, recall_score
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from joblib import dump, load
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
#Import Models
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
print("Numpy version: {}".format(np.__version__))
print("Pandas version: {}".format(pd.__version__))
import sklearn as sklearn
print("Sklearn version: {}".format(sklearn.__version__))
#import keras as keras
#print("Keras version: {}".format(keras.__version__))
#print("XBG Boost version: {}".format(xgboost.__version__))
import matplotlib as mpl
print ("Matplotlib version: {}".format(mpl.__version__))

Using TensorFlow backend.


Numpy version: 1.17.2
Pandas version: 0.25.1
Sklearn version: 0.21.3
Matplotlib version: 3.1.2
CPU times: user 3.9 s, sys: 1.39 s, total: 5.28 s
Wall time: 8.97 s


In [2]:
%%time
# Read csv file into dataframe
df = pd.read_csv('paysim_features.csv').drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 20 columns):
step              int64
amount            float64
ChangeOrig        float64
ChangeDest        float64
nameOrig          object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest          object
oldbalanceDest    float64
newbalanceDest    float64
isFraud           int64
isFlaggedFraud    int64
type_CASH_IN      int64
type_CASH_OUT     int64
type_DEBIT        int64
type_PAYMENT      int64
type_TRANSFER     int64
steptime          object
dayofweek         int64
timeofday         float64
dtypes: float64(8), int64(9), object(3)
memory usage: 970.9+ MB
CPU times: user 22.1 s, sys: 5.79 s, total: 27.9 s
Wall time: 30.5 s


In [3]:
df.describe()

,step,amount,ChangeOrig,ChangeDest,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,dayofweek,timeofday
count,"6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00","6,362,620.00"
mean,243.40,"179,861.90","21,230.56","124,294.73","833,883.10","855,113.67","1,100,701.67","1,224,996.40",0.00,0.00,0.22,0.35,0.01,0.34,0.08,3.12,15.32
std,142.33,"603,858.23","146,643.29","812,939.08","2,888,242.67","2,924,048.50","3,399,180.11","3,674,128.94",0.04,0.00,0.41,0.48,0.08,0.47,0.28,1.65,4.32
min,1.00,0.00,"-10,000,000.00","-13,060,826.21",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,156.00,"13,389.57","-10,150.44",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,12.00
50%,239.00,"74,871.94",0.00,0.00,"14,208.00",0.00,"132,705.66","214,661.44",0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,16.00
75%,335.00,"208,721.48",0.00,"149,105.43","107,315.18","144,258.41","943,036.71","1,111,909.25",0.00,0.00,0.00,1.00,0.00,1.00,0.00,4.00,19.00
max,743.00,"92,445,516.64","1,915,267.90","105,687,838.82","59,585,040.37","49,585,040.37","356,015,889.35","356,179,278.92",1.00,1.00,1.00,1.00,1.00,1.00,1.00,6.00,23.00


In [4]:
df.head()

,step,amount,ChangeOrig,ChangeDest,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_IN,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,steptime,dayofweek,timeofday
0,1,"9,839.64","-9,839.64",0.00,C1231006815,"170,136.00","160,296.36",M1979787155,0.00,0.00,0,0,0,0,0,1,0,1970-01-01 01:00:00,3,1.00
1,1,"1,864.28","-1,864.28",0.00,C1666544295,"21,249.00","19,384.72",M2044282225,0.00,0.00,0,0,0,0,0,1,0,1970-01-01 01:00:00,3,1.00
2,1,181.00,-181.00,0.00,C1305486145,181.00,0.00,C553264065,0.00,0.00,1,0,0,0,0,0,1,1970-01-01 01:00:00,3,1.00
3,1,181.00,-181.00,"-21,182.00",C840083671,181.00,0.00,C38997010,"21,182.00",0.00,1,0,0,1,0,0,0,1970-01-01 01:00:00,3,1.00
4,1,"11,668.14","-11,668.14",0.00,C2048537720,"41,554.00","29,885.86",M1230701703,0.00,0.00,0,0,0,0,0,1,0,1970-01-01 01:00:00,3,1.00


In [5]:
# Define X with selected columns, y as the binary variable to predict
X = df.drop(['isFraud' , 'isFlaggedFraud' , 'nameOrig' , 'nameDest' ,'steptime'] , axis = 1 ).values
y = df['isFraud'].values
# Train, Test, Split to segregate Training from Test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [6]:
X.shape

(6362620, 15)

In [12]:
%%time
# Setup the pipeline
steps = [('scaler', StandardScaler()),
         ('rfc', RandomForestClassifier())
         ]

pipeline = Pipeline(steps)

# Create the hyperparameter grid
param_grid = {'rfc__max_depth': [20, 30],
              'rfc__class_weight': [None],
              'rfc__criterion' : ['entropy', 'gini']}


# Instantiate the logistic regression classifier: logreg
#logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# Create train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.4)

# Instantiate the GridSearchCV object: logreg_cv
rfc = GridSearchCV(pipeline, param_grid, cv=5, verbose=True, n_jobs=-1)

# Fit it to the training data
rfc.fit(X_train, y_train)

rfc_predict = rfc.predict(X_test)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(rfc.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(rfc.best_score_))
print (confusion_matrix(y_test , rfc_predict))
print ("Accuracy score: ", accuracy_score(y_test , rfc_predict))
print("F1 score: ", f1_score(y_test , rfc_predict))
print("Precision score: " , precision_score(y_test , rfc_predict))
print("Recall score: " , recall_score(y_test , rfc_predict))
print("ROC-AUC score: " , roc_auc_score(y_test , rfc_predict))
print (classification_report(y_test , rfc_predict))
rfc

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed: 19.8min finished
C:\Users\19145\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Tuned Logistic Regression Parameter: {'rfc__class_weight': None, 'rfc__criterion': 'entropy', 'rfc__max_depth': 20}
Tuned Logistic Regression Accuracy: 0.999753919881163
[[1906316      35]
 [    414    2021]]
Accuracy score:  0.9997647719545303
F1 score:  0.9000222667557335
Precision score:  0.982976653696498
Recall score:  0.8299794661190965
ROC-AUC score:  0.9149805532180605
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.98      0.83      0.90      2435

    accuracy                           1.00   1908786
   macro avg       0.99      0.91      0.95   1908786
weighted avg       1.00      1.00      1.00   1908786

Wall time: 23min 7s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rfc',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                  

In [10]:
print (confusion_matrix(y_test , rfc_predict))
print ("Accuracy score: ", accuracy_score(y_test , rfc_predict))
print("F1 score: ", f1_score(y_test , rfc_predict))
print("Precision score: " , precision_score(y_test , rfc_predict))
print("Recall score: " , recall_score(y_test , rfc_predict))
print("ROC-AUC score: " , roc_auc_score(y_test , rfc_predict))
print (classification_report(y_test , rfc_predict))
rfc

[[1906308      43]
 [    447    1988]]
Accuracy score:  0.9997432923334517
F1 score:  0.890282131661442
Precision score:  0.9788281634662728
Recall score:  0.8164271047227926
ROC-AUC score:  0.9082022742704257
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.98      0.82      0.89      2435

    accuracy                           1.00   1908786
   macro avg       0.99      0.91      0.95   1908786
weighted avg       1.00      1.00      1.00   1908786



GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rfc',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                  

In [11]:
print("Tuned Logistic Regression Parameter: {}".format(rfc.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(rfc.best_score_))


Tuned Logistic Regression Parameter: {'rfc__class_weight': None, 'rfc__criterion': 'entropy', 'rfc__max_depth': 20}
Tuned Logistic Regression Accuracy: 0.9997494293680456


In [13]:
%%time
# Setup the pipeline
steps = [('scaler', StandardScaler()),
         ('rfc', RandomForestClassifier())
         ]

pipeline = Pipeline(steps)

# Create the hyperparameter grid
param_grid = {'rfc__n_estimators': [100],
              'rfc__max_depth': [20],
              'rfc__class_weight': [None],
              'rfc__criterion' : ['entropy']}


# Instantiate the logistic regression classifier: logreg
#logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# Create train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.4)

# Instantiate the GridSearchCV object: logreg_cv
rfc = GridSearchCV(pipeline, param_grid, cv=5, verbose=True, n_jobs=-1)

# Fit it to the training data
rfc.fit(X_train, y_train)

rfc_predict = rfc.predict(X_test)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(rfc.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(rfc.best_score_))
print (confusion_matrix(y_test , rfc_predict))
print ("Accuracy score: ", accuracy_score(y_test , rfc_predict))
print("F1 score: ", f1_score(y_test , rfc_predict))
print("Precision score: " , precision_score(y_test , rfc_predict))
print("Recall score: " , recall_score(y_test , rfc_predict))
print("ROC-AUC score: " , roc_auc_score(y_test , rfc_predict))
print (classification_report(y_test , rfc_predict))
rfc

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 27.1min finished


Tuned Logistic Regression Parameter: {'rfc__class_weight': None, 'rfc__criterion': 'entropy', 'rfc__max_depth': 20, 'rfc__n_estimators': 100}
Tuned Logistic Regression Accuracy: 0.9997561651377218
[[1906322      29]
 [    422    2013]]
Accuracy score:  0.9997637241681362
F1 score:  0.8992628992628993
Precision score:  0.9857982370225269
Recall score:  0.826694045174538
ROC-AUC score:  0.9133394164328935
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1906351
           1       0.99      0.83      0.90      2435

    accuracy                           1.00   1908786
   macro avg       0.99      0.91      0.95   1908786
weighted avg       1.00      1.00      1.00   1908786

Wall time: 46min 12s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rfc',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                  

In [ ]:
%%time
# Setup the pipeline
steps = [('scaler', StandardScaler()),
         ('rfc', RandomForestClassifier())
         ]

pipeline = Pipeline(steps)

# Create the hyperparameter grid
param_grid = {'rfc__n_estimators': [100],
              'rfc__max_depth': [20],
              'rfc__class_weight': [None, 'balanced'],
              'rfc__criterion' : ['entropy']}


# Instantiate the logistic regression classifier: logreg
#logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# Create train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.4)

# Instantiate the GridSearchCV object: logreg_cv
rfc = GridSearchCV(pipeline, param_grid, cv=5, verbose=True, n_jobs=-1)

# Fit it to the training data
rfc.fit(X_train, y_train)

rfc_predict = rfc.predict(X_test)

# Print the optimal parameters and best score
print("Tuned Logistic Regression Parameter: {}".format(rfc.best_params_))
print("Tuned Logistic Regression Accuracy: {}".format(rfc.best_score_))
print (confusion_matrix(y_test , rfc_predict))
print ("Accuracy score: ", accuracy_score(y_test , rfc_predict))
print("F1 score: ", f1_score(y_test , rfc_predict))
print("Precision score: " , precision_score(y_test , rfc_predict))
print("Recall score: " , recall_score(y_test , rfc_predict))
print("ROC-AUC score: " , roc_auc_score(y_test , rfc_predict))
print (classification_report(y_test , rfc_predict))
rfc

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
